# Fudan PRML 23Spring Assignment2


<font color='red'>*Your Name, Student ID and Date: [Name], [Student ID], [Date]*</font>

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission.*


<font color='red' size = 3> *DDL: 2023.06.11 23:59* </font>

## 1. FDUNN: your toy torch-like deep learning library (60 points)

In this assignment, you will fist implement your own torch-like deep learning library with `numpy`, named `fdunn`.

PyTorch: [Link](https://pytorch.org/)

In [ ]:
# setup code, auto reload your .py file
%load_ext autoreload
%autoreload 2

In [ ]:
# numpy
import numpy as np
np.random.seed(233)

In [ ]:
# matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

You have to impletement several standard deep neural network modules in the `./fdunn` folder:
1.   linear/conv/pooling
2.   activation
3.   loss
4.   optim
5.   trainer

We have written most of the code for you already, and you only need to fill in the most essential parts. We have also prepared several test cases for you to check if your code works correctly.

Furthermore, you can also test the accuracy of your code by comparing its output with the output of sk-learn.

In [ ]:
from .fdunn import *


## 2. Deep Learning with Image/Text Data (40 points)

Use your fdunn lib to perform a task like 'image classification' or 'text classification'.

Some related datasets:
- MNIST: http://yann.lecun.com/exdb/mnist/
- CIFAR10: https://www.cs.toronto.edu/~kriz/cifar.html
- ChnSentiCorp: https://raw.githubusercontent.com/SophonPlus/ChineseNlpCorpus/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv